# Machine learning algorithm

In [2]:
# IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_val_score, validation_curve, train_test_split

In [3]:
# LOAD THE DATA
# use pandas to read data as table
# index is already specified in *.txt file -> use first column as index column
artworks = pd.read_table("beautified_boxes_numeric.txt", index_col=0)
artworks

,Artwork-Id,type,district,environment,countArtists,experience,replaced,content,userRating,approval
0,32048698,5,3,3,2,1,7,2,4,1
1,39800694,5,3,3,3,2,4,1,2,0
2,80318972,4,3,2,3,2,8,2,4,0
3,74478002,4,3,3,4,1,3,2,4,0
4,71449602,5,2,4,3,2,4,2,0,0
...,...,...,...,...,...,...,...,...,...,...
745,68311402,5,4,4,2,1,6,1,3,1
746,22390811,5,1,3,2,2,6,2,5,1
747,19230282,5,5,3,1,3,6,2,5,1
748,29644044,5,3,4,2,3,8,2,3,1


In [29]:
# SET FEATURES AND LABEL

# set features and label
# features (X): "type", "district", "environment", "countArtists", "experience", "replaced", "content", "userRating"
# label (y): "approval"
# do not use for classification: "Artwork_Id"
X = artworks[["type", "district", "environment", "countArtists", "experience", "replaced", "content", "userRating"]]
y = artworks["approval"]


In [ ]:
# DO TRAIN-TEST-SPLIT

# train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [30]:
# CREATE CLASSIFIER OBJECT
# decision: knn classifier
# find out the optimum value for k using a loop and a plot.
# optional: also do a tree classifier to compare. but first finish the knn!
knn = KNeighborsClassifier(n_neighbors=5)

In [31]:
# TRAIN THE CLASSIFIER
# based on previous results, decide for one and train one final classifier using this special value of k
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [32]:
# EVALUATE ACCURACY OF THE CLASSIFIER
# obligatory: calculate score over test set
# optional: do cross validation
# optional: evaluate proportion of the train/test split proportion
knn.score(X_test, y_test)

0.7925531914893617

In [ ]:
# USE THE TRAINED CLASSIFIER TO CLASSIFY NEW PREVIOUSLY UNSEEN OBJECTS (optional)

In [ ]:
"""
Plotting the decision boundaries may not be possible since it is from the course's code library and may be specialised on the fruit data.

"""